In [219]:
import pandas as pd
import numpy as np
from IPython.display import display
pd.options.display.max_rows = 13
import math

In [220]:
import re 
import sys

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    return text_subbed

def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_arroba(lista):
    arroba = "@"
    for i in lista: 
        if arroba in i:
            lista.remove(i)
    return lista
def remove_link(lista):
    link = "/"
    for i in lista: 
        if link in i:
            lista.remove(i)
    return lista

In [221]:
#treinamento
dados = pd.read_excel('Iphone_CDados.xlsx')

#teste
excel = pd.ExcelFile('Iphone_CDados.xlsx')
teste = pd.read_excel(excel, 'Teste')
teste

Teste  Filtro
0   vei o cllr da minha mae é ruim dms, iphone so ...       1
1   o ipad q um dia eu vo comprar ta mais barato q...       1
2   agora entendi porque os xiaominions são chatos...       1
3   alguém conhece quem vende cabo, de preferência...       1
4   @gcamaraa bateria de iphone é mais feias q bat...       1
..                                                ...     ...
69  rt @turnes_caroline: gente meu celular foi rou...       0
70                    meu iphone ja era , que odio 😑😣       0
71  @luizzzlima_ o meu é iphone e pega, mas o da m...       0
72      alguém com iphone pode tirar print do meu fc?       0
73  amanhã não há iphone 12.😑😅 https://t.co/w6qa1q...       0

[74 rows x 2 columns]

In [222]:
dados

Treinamento  Filtro
0    @kayllanefer bateria de iphone é uma merda, ja...       1
1             cabo de iphone eh feito pra estragar pqp       1
2    @johnpvx melhor iphone de sempre. não me arrep...       1
3    alguém querendo comprar um cabo/carregador de ...       1
4    to sem carregador já tava sofrendo aqui daí le...       1
..                                                 ...     ...
328              acho que amanhã lança o iphone 12 👀👀👀       0
329  @pirucaking @paulocoelho não dá pra levar a sé...       0
330  to querendo comprar o iphone 11pro max, para m...       0
331  triste tirar foto só com iphone 😔 pois é a úni...       0
332  um amigo meu vai se alistar na cruz vermelha e...       0

[333 rows x 2 columns]

In [223]:
Relevante = dados["Filtro"].isin(["1"])
Irrelevante = dados["Filtro"].isin(["0"])
Relevante = dados.loc[Relevante, :]
Irrelevante = dados.loc[Irrelevante, :]


total_frases_relevantes = Relevante.count()[0]
total_frases_irrelevantes = Irrelevante.count()[0]

In [224]:
#limpando texto e fazendo lista

Treino_Relevante = Relevante["Treinamento"]
Treino_Irrelevante = Irrelevante["Treinamento"]

def make_list(Treino_Relevante):
    
    treinamento_relevante = []

    #fazendo lista
    for i in Treino_Relevante:
        treinamento_relevante.append(i)
    
    coma = "'"
    #transformar em texto
    treinamento_relevante = coma.join(treinamento_relevante)

    #função que limpa
    limpa = cleanup(treinamento_relevante)
    no_emoji = remove_emoji(limpa)

    #lista por palavra
    minusculo = no_emoji.lower()
    palavras_relevantes = minusculo.split()

    lista = remove_arroba(palavras_relevantes)
    lista = remove_link(lista)
    
    return lista

todas_palavras_relevantes = make_list(Treino_Relevante)
todas_palavras_irrelevantes = make_list(Treino_Irrelevante)

In [225]:
#pdSeries
serie_relevante = pd.Series(todas_palavras_relevantes)
serie_irrelevante = pd.Series(todas_palavras_irrelevantes)

In [226]:
#tabelas frequencia relativas
tabela_relevantes = serie_relevante.value_counts()
tabela_irrelevantes = serie_irrelevante.value_counts()
Nr = sum(tabela_relevantes)
Ni = sum(tabela_irrelevantes)

In [227]:
def limpa(tweet):
    pri = cleanup(tweet)
    seg = remove_emoji(pri)
    
    return seg


In [228]:
#P(frase/relevante)
def P_tweet_dado_relevante(lista):
    
    prob_tweet_dado_relevante = 1
    alpha = 1
    V = 100000
    prob_tweets = []
    
    for tweet in lista:
        
        
        prob_tweet_dado_relevante = 1
        tweet_minusculo = tweet.lower()
        tweet_limpo = limpa(tweet_minusculo) 
        lista_limpo = tweet_limpo.split()
        lista_limpo = remove_arroba(lista_limpo)
        lista_limpo = remove_link(lista_limpo)
    
        for palavra in lista_limpo:
        
            if palavra in tabela_relevantes:
                prob_palavra_dado_relevante = (tabela_relevantes[palavra] + alpha)/ (Nr+(V * alpha))
        
            else:
            
                prob_palavra_dado_relevante = alpha/(Nr + (alpha * V))
            
            prob_tweet_dado_relevante += math.log(prob_palavra_dado_relevante)
        
        prob_tweets.append(prob_tweet_dado_relevante)
        
    return prob_tweets


#P(frase/irrelevante)
def P_tweet_dado_irrelevante(lista):
    
    prob_tweet_dado_irrelevante = 1
    alpha = 1
    V = 100000
    
    prob_tweets = []
    
    for tweet in lista:
        
        
        prob_tweet_dado_irrelevante = 1
        tweet_minusculo = tweet.lower()
        tweet_limpo = limpa(tweet_minusculo)
        lista_limpo = tweet_limpo.split()
        lista_limpo = remove_arroba(lista_limpo)
        lista_limpo = remove_link(lista_limpo)
        
        for palavra in lista_limpo:
        
            if palavra in tabela_irrelevantes:
                prob_palavra_dado_irrelevante = (tabela_irrelevantes[palavra] + alpha)/ (Ni+(V * alpha))
        
            else:
            
                prob_palavra_dado_irrelevante = alpha/(Ni + (alpha * V))
            
            prob_tweet_dado_irrelevante += math.log(prob_palavra_dado_irrelevante)
        
        prob_tweets.append(prob_tweet_dado_irrelevante)
        
    return prob_tweets

    

In [229]:
#prob relevante
P_R = total_frases_relevantes/(total_frases_relevantes+total_frases_irrelevantes)
#prob irrelevante
P_I = 1 - P_R

In [230]:
teste_relevante = teste["Filtro"].isin(["1"])
teste_irrelevante = teste["Filtro"].isin(["0"])
Teste_Relevante = teste.loc[teste_relevante, :]
Teste_Irrelevante = teste.loc[teste_irrelevante, :]



def lista_treino(data_frame):
    
    lista_teste = []
    
    for i in data_frame["Teste"]:
        lista_teste.append(i)
        
    return lista_teste

lista_teste = lista_treino(teste)

In [231]:
Prob_tweet_dado_relevante = P_tweet_dado_relevante(lista_teste)
Prob_tweet_dado_irrelevante = P_tweet_dado_irrelevante(lista_teste)

In [234]:
Prob_relevante_dado_tweet = [i + math.log(P_R) for i in Prob_tweet_dado_relevante]
Prob_irrelevante_dado_tweet = [i + math.log(P_I) for i in Prob_tweet_dado_irrelevante]

In [235]:
valor_maior = []

for i in range(len(Prob_relevante_dado_tweet)):
    if Prob_relevante_dado_tweet[i] > Prob_irrelevante_dado_tweet[i]:
        valor_maior.append(1)
    else:
        valor_maior.append(0)
        
sum(valor_maior)

10